In [9]:
import pandas as pd
import numpy as np

# -------------------------------
# Config
# -------------------------------
N_ACCOUNTS = 1200
PRODUCTS = ["P01", "P02", "P03", "P04"]
START_DATE = "2023-01-01"
END_DATE = "2024-12-01"
SEED = 42

np.random.seed(SEED)

months = pd.date_range(START_DATE, END_DATE, freq="MS")

rows = []

for acc in range(1, N_ACCOUNTS + 1):
    account_id = f"A{acc:05d}"

    n_products = np.random.randint(1, len(PRODUCTS) + 1)
    account_products = np.random.choice(PRODUCTS, n_products, replace=False)

    # Account-level behavior
    base_revenue = np.random.lognormal(mean=7.5, sigma=0.6)
    trend_type = np.random.choice(
        ["growth", "decline", "flat", "churn"],
        p=[0.35, 0.25, 0.25, 0.15]
    )

    for prod in account_products:
        product_multiplier = np.random.uniform(0.4, 1.2)
        revenue = base_revenue * product_multiplier

        churn_month = np.random.choice(
            months[10:], 1
        )[0] if trend_type == "churn" else None

        for m in months:
            if churn_month is not None and m >= churn_month:
                revenue = revenue * np.random.uniform(0.0, 0.1)

            if trend_type == "growth":
                revenue *= np.random.uniform(1.00, 1.06)
            elif trend_type == "decline":
                revenue *= np.random.uniform(0.94, 1.00)
            elif trend_type == "flat":
                revenue *= np.random.uniform(0.98, 1.02)

            # Seasonality
            seasonal = 1 + 0.15 * np.sin(2 * np.pi * m.month / 12)
            revenue *= seasonal

            revenue_month = max(0, np.random.normal(revenue, revenue * 0.15))

            rows.append({
                "MONTH_ID": m,
                "END_SHIPTO_CUSTOMER": account_id,
                "PRODUCT": prod,
                "REVENUE": round(revenue_month, 2),

                # Commercial signals
                "BACKLOG_ORDERS": int(max(0, revenue_month / 1000 + np.random.poisson(1))),
                "ORDERED_QUANTITY": int(max(0, revenue_month / 120 + np.random.poisson(3))),
                "FACE_TO_FACE_CALLS_OPP_COUNT": np.random.poisson(1),
                "AGGREGATED_SOURCE_OPP_COUNT": np.random.poisson(2),
            })

# -------------------------------
# Build DataFrame
# -------------------------------
Churn_PREVENA = pd.DataFrame(rows)

# -------------------------------
# Revenue Deciles & Segments
# -------------------------------
Churn_PREVENA["REVENUE_DECILE"] = (
    Churn_PREVENA.groupby("MONTH_ID")["REVENUE"]
    .transform(lambda x: pd.qcut(x.rank(method="first"), 10, labels=False) + 1)
)

Churn_PREVENA["REVENUE_SEGMENT"] = pd.cut(
    Churn_PREVENA["REVENUE_DECILE"],
    bins=[0, 3, 7, 10],
    labels=["LOW", "MID", "HIGH"]
)

print("Synthetic dataset shape:", Churn_PREVENA.shape)
print(Churn_PREVENA.head())


Synthetic dataset shape: (73536, 10)
    MONTH_ID END_SHIPTO_CUSTOMER PRODUCT  REVENUE  BACKLOG_ORDERS  \
0 2023-01-01              A00001     P02  1854.60               4   
1 2023-02-01              A00001     P02  1471.94               1   
2 2023-03-01              A00001     P02  2166.24               2   
3 2023-04-01              A00001     P02  2060.54               3   
4 2023-05-01              A00001     P02  2689.50               4   

   ORDERED_QUANTITY  FACE_TO_FACE_CALLS_OPP_COUNT  \
0                18                             0   
1                15                             1   
2                21                             0   
3                20                             1   
4                23                             0   

   AGGREGATED_SOURCE_OPP_COUNT  REVENUE_DECILE REVENUE_SEGMENT  
0                            2               7             MID  
1                            0               5             MID  
2                            0    

In [2]:
df = Churn_PREVENA.copy()

rename_map = {
    "MONTH_ID": "data_date",
    "END_SHIPTO_CUSTOMER": "account_id",
    "PRODUCT": "PRODUCT_ID",
    "REVENUE": "revenue"
}

In [13]:
# ============================================================
# ACCOUNT × PRODUCT CLV → CHURN → UPSELL PIPELINE (FINAL FIX)
# ============================================================

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, precision_score, recall_score

import warnings
warnings.filterwarnings("ignore")

# ============================================================
# STEP 1: BASE DATA PREPARATION
# ============================================================

df_input = Churn_PREVENA.copy()

rename_map = {
    "MONTH_ID": "data_date",
    "END_SHIPTO_CUSTOMER": "account_id",
    "PRODUCT": "PRODUCT_ID",
    "REVENUE": "revenue"
}
df_input.rename(columns=rename_map, inplace=True)
df_input["data_date"] = pd.to_datetime(df_input["data_date"])

# ============================================================
# STEP 2: PIPELINE A — CLV
# ============================================================

class CLVChurnPipelineACCPROD:

    def __init__(self, df):
        self.df = df.copy()

    def _build_monthly_panel(self):
        df_month = (
            self.df
            .groupby([
                "account_id",
                "PRODUCT_ID",
                self.df["data_date"].dt.to_period("M")
            ])["revenue"]
            .sum()
            .reset_index()
        )

        df_month["data_date"] = df_month["data_date"].dt.to_timestamp()
        df_month.rename(columns={"revenue": "revenue_month"}, inplace=True)

        panels = []

        for (acct, prod), g in df_month.groupby(["account_id", "PRODUCT_ID"]):
            g = g.sort_values("data_date").set_index("data_date")
            idx = pd.date_range(g.index.min(), g.index.max(), freq="MS")
            g = g.reindex(idx)
            g["account_id"] = acct
            g["PRODUCT_ID"] = prod
            g["revenue_month"] = g["revenue_month"].fillna(0)
            g = g.reset_index().rename(columns={"index": "data_date"})
            panels.append(g)

        return pd.concat(panels, ignore_index=True)

    def run(self):
        panel = self._build_monthly_panel()

        def compute(g):
            g = g.reset_index(drop=True)

            g["CLV_6M_Current_ACCPROD"] = g["revenue_month"].rolling(6, 1).sum()
            g["CLV_6M_Prior_ACCPROD"] = g["revenue_month"].shift(6).rolling(6, 1).sum()
            g["CLV_12M_Current_ACCPROD"] = g["revenue_month"].rolling(12, 1).sum()
            g["CLV_12M_Prior_ACCPROD"] = g["revenue_month"].shift(12).rolling(12, 1).sum()

            eps = 1e-6
            g["CLV_Ratio_6M_ACCPROD"] = (
                g["CLV_6M_Current_ACCPROD"] / (g["CLV_6M_Prior_ACCPROD"] + eps)
            ).clip(0, 3)

            g["CLV_Ratio_12M_ACCPROD"] = (
                g["CLV_12M_Current_ACCPROD"] / (g["CLV_12M_Prior_ACCPROD"] + eps)
            ).clip(0, 3)

            last = g.iloc[-1]

            return last[
                [
                    "account_id",
                    "PRODUCT_ID",
                    "CLV_6M_Current_ACCPROD",
                    "CLV_6M_Prior_ACCPROD",
                    "CLV_Ratio_6M_ACCPROD",
                    "CLV_12M_Current_ACCPROD",
                    "CLV_12M_Prior_ACCPROD",
                    "CLV_Ratio_12M_ACCPROD",
                ]
            ]

        clv = (
            panel
            .groupby(["account_id", "PRODUCT_ID"])
            .apply(compute)
            .reset_index(drop=True)
        )

        clv["CLV_CHURN_SCORE_ACCPROD"] = (
            0.6 * (1 - clv["CLV_Ratio_6M_ACCPROD"].clip(0, 1)) +
            0.4 * (1 - clv["CLV_Ratio_12M_ACCPROD"].clip(0, 1))
        ).clip(0, 1)

        return clv

# Run Pipeline A
pipeA = CLVChurnPipelineACCPROD(df_input)
df_clv = pipeA.run()

# IMPORTANT FIX: merge WITHOUT data_date collision
df_model_base = df_input.merge(
    df_clv,
    on=["account_id", "PRODUCT_ID"],
    how="left"
)




In [16]:
display(df_clv.head(4))


23,account_id,PRODUCT_ID,CLV_6M_Current_ACCPROD,CLV_6M_Prior_ACCPROD,CLV_Ratio_6M_ACCPROD,CLV_12M_Current_ACCPROD,CLV_12M_Prior_ACCPROD,CLV_Ratio_12M_ACCPROD,CLV_CHURN_SCORE_ACCPROD
0,A00001,P01,20956.25,25365.58,0.826169,46321.83,35647.29,1.299449,0.104299
1,A00001,P02,17873.19,16478.82,1.084616,34352.01,25247.57,1.360607,0.000000
2,A00001,P03,40576.51,41888.69,0.968675,82465.20,61532.10,1.340198,0.018795
3,A00002,P03,1606.83,2384.95,0.673737,3991.78,5763.36,0.692613,0.318712
